# Топ уникальных событий
Пример выгрузки топа уникальных событий по рейтингу (для программ, выходивших несколько раз за анализируемый период, отобран только один выход, получивший наибольший рейтинг)

## Описание задачи и условий расчета
- Период: 31.12.2022
- ЦА: Все 4+
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index
- Программы: жанр - все, кроме Прочее, длительность 5+ минут
- Статистики: Rtg%, Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники
Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
#Обратимся к каталогу жанров программ
cats.get_tv_program_type()

#id жанра Прочее - 9

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2022-12-31', '2022-12-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = None

# Указываем фильтр программ: жанр - все, кроме Прочее и продолжительность 5+ минут (300 секунд)  
program_filter = 'programTypeId != 9 and programDuration >= 300'

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем список срезов
slices = ['programSpotId', #Программа ID выхода, обязательный атрибут! 
          'programId', #Программа ID (понадобится для фильтрации уникальных программ)
          'programName', #Программа
          'researchDate', #Дата, обязательный атрибут! 
          'researchWeekDay', #День недели
          'programStartTime', #Программа время начала
          'programFinishTime', #Программа время окончания
          'programCategoryName', #Категория
          'programTypeName', #Жанр,
          'tvCompanyName' #Телекомпания
         ]
# Указываем список статистик для расчета
statistics = ['RtgPer', 'Share']

# Задаем условия сортировки посчитанного отчета: RtgPer (по убыванию)
sortings = {'RtgPer':'DESC'}

# Задаем опции расчета
options = {
    "kitId": 1 #TV Index Russia all
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_simple_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task))
df

Для программ, выходивших несколько раз за период оставляем только выходы с наибольшим рейтингом

In [ ]:
# для каждой пары канал-программа удаляем дублирующие значения (если есть) и оставляем первое с наибольшим значением рейтинга
df.drop_duplicates(['tvCompanyName','programId'], keep='first', inplace=True)

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('01_top_unique_unlim')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)